In [1]:
from langchain_community.document_loaders import PyPDFLoader # type: ignore
from langchain.text_splitter import RecursiveCharacterTextSplitter # type: ignore
from langchain_community.vectorstores import FAISS # type: ignore
from langchain_community.embeddings import HuggingFaceEmbeddings # type: ignore
from langchain_community.chat_models import ChatOllama # type: ignore
from langchain_core.prompts import ChatPromptTemplate # type: ignore
from langchain.chains.combine_documents import create_stuff_documents_chain # type: ignore
from langchain.chains import create_retrieval_chain # type: ignore

In [2]:
# 1. 문서 로드 (Load)
# 답변의 근거가 될 PDF 문서를 로드합니다.
from pathlib import Path
pdf_path = Path("/Users/snu.sim/git/RAG_test/The Ghost in the Machine.pdf")
loader = PyPDFLoader(str(pdf_path))

docs = loader.load()

In [3]:
# 2. 문서 분할 (Split)
# 문서를 검색하기 좋은 크기의 여러 조각(청크)으로 나눕니다.
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
split_docs = text_splitter.split_documents(docs)

In [5]:
# 3. 임베딩 & 벡터 스토어 생성 (Store)
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2", # Huggingface 모델명
    model_kwargs={'device': 'cpu'},
    encode_kwargs={'normalize_embeddings': True}
)

'(ProtocolError('Connection aborted.', ConnectionResetError(54, 'Connection reset by peer')), '(Request ID: bafdd925-a450-408a-b135-abcfe117fc23)')' thrown while requesting HEAD https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2/resolve/main/./modules.json
Retrying in 1s [Retry 1/5].


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [6]:
# 분할된 문서를 임베딩하여 FAISS 벡터 스토어에 저장합니다. (메모리 기반)
vectorstore = FAISS.from_documents(split_docs, embeddings)

In [7]:
# 4. 검색기(Retriever) 생성
# 벡터 스토어에서 관련 문서를 검색하는 역할을 합니다.
retriever = vectorstore.as_retriever()

In [8]:
# 5. LLM 로드 (Ollama 연동)
# 로컬에서 실행 중인 Ollama의 모델을 LangChain에서 사용할 수 있도록 설정합니다.
llm = ChatOllama(model="gemma3:4b") 

/var/folders/2q/_8m2fvxd4pv04q2b3gk8j0bm0000gn/T/ipykernel_19744/896109414.py:3: LangChainDeprecationWarning: The class `ChatOllama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import ChatOllama``.
  llm = ChatOllama(model="gemma3:4b")


In [9]:
# 6. 프롬프트 정의
# LLM에게 질문과 함께 검색된 문서를 어떻게 활용할지 지시하는 템플릿입니다.
prompt = ChatPromptTemplate.from_template("""
Answer the following question based only on the provided context.

<context>
{context}
</context>

Question: {input}
""")

In [ ]:
# 7. RAG 체인 생성
# LangChain Expression Language (LCEL)을 사용하여 체인을 구성합니다.
document_chain = create_stuff_documents_chain(llm, prompt)
retrieval_chain = create_retrieval_chain(retriever, document_chain)

# 8. 체인 실행 및 질문
response = retrieval_chain.invoke({"input": "Adapter BERT의 저자가 누구야?"})

# 답변 출력
# 답변의 근거가 된 문서(Context) 출력
print("---------- 검색된 근거 문서 ----------\n")
for i, doc in enumerate(response["context"]):
    print(f"문서 #{i+1}:\n")
    print(doc.page_content)
    print("\n------------------------------------\n")

# 최종 답변 출력
print("---------- AI 최종 답변 ----------\n")
print(response["answer"])